# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [24]:
# Setup - Run this cell first
import polars as pl

# Define a more complete schema for weather to ensure correct data types
weather_schema = {
    "origin": pl.Utf8,
    "year": pl.Int64,
    "month": pl.Int64,
    "day": pl.Int64,
    "hour": pl.Int64,
    "temp": pl.Float64,
    "dewp": pl.Float64,
    "humid": pl.Float64,
    "wind_dir": pl.Int64,
    "wind_speed": pl.Float64,
    "wind_gust": pl.Float64,
    "precip": pl.Float64,
    "pressure": pl.Float64,
    "visib": pl.Float64, # Changed from pl.Int64 to pl.Float64
    "time_hour": pl.Utf8, # Read as Utf8 initially, then parse to Datetime
}

# Load data from local files
airlines = pl.read_csv('nyc_airlines.csv', null_values='NA')
airports = pl.read_csv('nyc_airports.csv', null_values='NA')
flights = pl.read_csv('nyc_flights.csv', null_values='NA')
planes = pl.read_csv('nyc_planes.csv', null_values='NA')
weather = pl.read_csv('nyc_weather.csv', schema=weather_schema, null_values='NA')

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%SZ", strict=False))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%SZ", strict=False))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-2643517061.py:34: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [25]:
# Write your SQL query here
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
""")

print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [26]:
# Write your SQL query here
result = ctx.execute("""
SELECT dest, COUNT(*) AS num_flights
FROM flights
GROUP BY dest
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [27]:
# Write your SQL query here
result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour           │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                 │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs]        │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …                   │
│ 2013 ┆ 9     ┆ 30  ┆ 1

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [28]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    origin,
    AVG(dep_delay) as avg_delay
FROM flights
WHERE dep_delay IS NOT NULL
GROUP BY origin
ORDER BY avg_delay DESC
""")

print(result)

shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [29]:
# First, let's check what columns are available
result = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")
# print(result)

# Now write your query to find busiest month
result = ctx.execute("""
SELECT month, COUNT(*) AS num_flights
FROM flights
GROUP BY month
ORDER BY num_flights DESC
LIMIT 1
""")

print(result)

shape: (1, 2)
┌───────┬─────────────┐
│ month ┆ num_flights │
│ ---   ┆ ---         │
│ i64   ┆ u32         │
╞═══════╪═════════════╡
│ 7     ┆ 29425       │
└───────┴─────────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [30]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    carrier,
    CAST(SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) AS FLOAT) * 100 / COUNT(*) AS on_time_rate
FROM flights
WHERE dep_delay IS NOT NULL
GROUP BY carrier
ORDER BY on_time_rate DESC
""")

print(result)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ HA      ┆ 92.982456    │
│ US      ┆ 87.822674    │
│ AS      ┆ 86.797753    │
│ AA      ┆ 84.071293    │
│ DL      ┆ 83.681246    │
│ …       ┆ …            │
│ FL      ┆ 73.32915     │
│ WN      ┆ 73.102706    │
│ F9      ┆ 71.847507    │
│ YV      ┆ 71.376147    │
│ EV      ┆ 69.538126    │
└─────────┴──────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [31]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    a.name AS airline_name,
    f.flight,
    f.origin,
    f.dest
FROM flights AS f
JOIN airlines AS a ON f.carrier = a.carrier
LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [32]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    AVG(2013 - p.year) AS average_plane_age
FROM flights AS f
JOIN planes AS p ON f.tailnum = p.tailnum
WHERE p.year IS NOT NULL
GROUP BY f.carrier
ORDER BY average_plane_age DESC
""")

print(result)

shape: (16, 2)
┌─────────┬───────────────────┐
│ carrier ┆ average_plane_age │
│ ---     ┆ ---               │
│ str     ┆ f64               │
╞═════════╪═══════════════════╡
│ MQ      ┆ 35.319            │
│ AA      ┆ 25.869426         │
│ DL      ┆ 16.372169         │
│ UA      ┆ 13.207691         │
│ FL      ┆ 11.385829         │
│ …       ┆ …                 │
│ B6      ┆ 6.686702          │
│ F9      ┆ 4.87874           │
│ VX      ┆ 4.473643          │
│ AS      ┆ 3.33662           │
│ HA      ┆ 1.548387          │
└─────────┴───────────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [33]:
# First, explore the weather table structure
result = ctx.execute("""
    SELECT *
    FROM weather
    LIMIT 5
""")
# print(result)

# Now write your join query
result = ctx.execute("""
SELECT
    f.* -- Select all columns from the flights table
FROM flights AS f
JOIN weather AS w ON f.time_hour = w.time_hour AND f.origin = w.origin -- Join on timestamp and origin airport
WHERE f.dep_delay > 30
  AND (w.wind_speed > 20 OR w.precip > 0.1)
""")

print(result)

shape: (4_938, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour           │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                 │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs]        │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 2217     ┆ … ┆ 1005     ┆ 21   ┆ 30     ┆ 2013-01-02 02:00:00 │
│ 2013 ┆ 1     ┆ 1   ┆ 2326     ┆ … ┆ 2248     ┆ 21   ┆ 30     ┆ 2013-01-02 02:00:00 │
│ 2013 ┆ 1     ┆ 4   ┆ 1238     ┆ … ┆ 820      ┆ 11   ┆ 59     ┆ 2013-01-04 16:00:00 │
│ 2013 ┆ 1     ┆ 4   ┆ 1400     ┆ … ┆ 212      ┆ 13   ┆ 20     ┆ 2013-01-04 18:00:00 │
│ 2013 ┆ 1     ┆ 4   ┆ 1455     ┆ … ┆ 187      ┆ 14   ┆ 22     ┆ 2013-01-04 19:00:00 │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …                   │
│ 2013 ┆ 9     ┆ 22  ┆ 1

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [34]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    p.manufacturer,
    p.model,
    COUNT(f.flight) AS num_flights
FROM flights AS f
JOIN planes AS p ON f.tailnum = p.tailnum
GROUP BY p.manufacturer, p.model
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬─────────────┐
│ manufacturer                  ┆ model           ┆ num_flights │
│ ---                           ┆ ---             ┆ ---         │
│ str                           ┆ str             ┆ u32         │
╞═══════════════════════════════╪═════════════════╪═════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278       │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027       │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716       │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553       │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051       │
│ BOEING                        ┆ 737-824         ┆ 13809       │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807       │
│ BOEING                        ┆ 737-7H4         ┆ 10389       │
│ BOEING                        ┆ 757-222         ┆ 9150        │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88           ┆ 8932     

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [42]:
# Solve Exercise 4.2 using Polars DataFrame API

# Join flights with airports for origin airport names
flights_with_origin_names = flights.join(
    airports.select(["faa", "name"]),
    left_on="origin",
    right_on="faa",
    how="left"
).rename({"name": "origin_airport_name"})

# Join the result with airports again for destination airport names
flights_with_airport_names = flights_with_origin_names.join(
    airports.select(["faa", "name"]),
    left_on="dest",
    right_on="faa",
    how="left"
).rename({"name": "dest_airport_name"})

# Perform the aggregation
polars_result_4_2 = (
    flights_with_airport_names
    .group_by(["origin_airport_name", "dest_airport_name"])
    .agg([
        pl.len().alias("total_flights"), # Changed from pl.count() to pl.len()
        pl.col("dep_delay").mean().alias("average_dep_delay"),
        ((pl.col("dep_delay") > 30).sum().cast(pl.Float64) * 100 / pl.len()).alias("percentage_delayed_more_than_30_min") # Sum boolean expression and changed count() to len()
    ])
    .sort("total_flights", descending=True)
    .limit(10)
)

print("Result of Exercise 4.2 using Polars DataFrame API:")
display(polars_result_4_2)

Result of Exercise 4.2 using Polars DataFrame API:


origin_airport_name,dest_airport_name,total_flights,average_dep_delay,percentage_delayed_more_than_30_min
str,str,u32,f64,f64
"""John F Kennedy Intl""","""Los Angeles Intl""",11262,8.522508,9.829515
"""La Guardia""","""Hartsfield Jackson Atlanta Int…",10263,11.448621,12.247881
"""La Guardia""","""Chicago Ohare Intl""",8857,10.740758,13.345377
"""John F Kennedy Intl""","""San Francisco Intl""",8204,11.952691,12.116041
"""La Guardia""","""Charlotte Douglas Intl""",6168,8.965321,11.948768
"""Newark Liberty Intl""","""Chicago Ohare Intl""",6100,14.644163,16.0
"""John F Kennedy Intl""",null,6049,9.597871,11.919326
"""John F Kennedy Intl""","""General Edward Lawrence Logan …",5898,11.694953,13.767379
"""La Guardia""","""Miami Intl""",5781,7.361747,9.462031


## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [44]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result_2_1 = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result_2_1 = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("Exercise 2.1 - SQL Result:")
print(sql_result_2_1)
print("\nExercise 2.1 - Polars Result:")
print(polars_result_2_1)

# Now implement one of your own queries in Polars below:
# Let's implement Exercise 1.2: Find the top 10 destinations by number of flights

# SQL version of Exercise 1.2 (for reference)
sql_result_1_2 = ctx.execute("""
SELECT dest, COUNT(*) AS num_flights
FROM flights
GROUP BY dest
ORDER BY num_flights DESC
LIMIT 10
""")

# Polars version of Exercise 1.2
polars_result_1_2 = (
    flights
    .group_by("dest")
    .agg(pl.len().alias("num_flights"))
    .sort("num_flights", descending=True)
    .limit(10)
)

print("\nExercise 1.2 - SQL Result:")
print(sql_result_1_2)
print("\nExercise 1.2 - Polars Result:")
print(polars_result_1_2)

Exercise 2.1 - SQL Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Exercise 2.1 - Polars Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Exercise 1.2 - SQL Result:
shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘

Exercise 1.2 - Polars Result:
shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights